** Read stuff from your own gdrive**

In [0]:
!pip install torch<=1.2.0
!pip install torchtext==0.4
%matplotlib inline

/bin/bash: =1.2.0: No such file or directory
     |████████████████████████████████| 61kB 2.8MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
!pip install PyDrive



In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [139]:

# Import the corpus data

from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


Constants

In [0]:
MY_HOME = "My Drive/nlp"


In [141]:
import numpy as np
def one_hot(a, num_classes):
  a = np.array([a])
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

print(one_hot(1, 2))

[0. 1.]


In [142]:
from sklearn.neural_network import MLPClassifier
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.metrics import classification_report

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

from sklearn.metrics.pairwise import cosine_similarity


seed = 123
np.random.seed(seed)

def to_tdidf():
    print("==========> Data Preprocessing")
    data = pd.read_pickle(os.path.join("drive/My Drive/nlp", "preprocessed_claim_body.pkl"))

    print(data.head())
    lim_unigram = 500
    # no more ram, limit this
    vectorizer_claim = TfidfVectorizer(max_features=lim_unigram)
    vectorizer_claim.fit(data['claim'][0:90])
    vectorizer_body = TfidfVectorizer(max_features=lim_unigram)
    vectorizer_body.fit(data['body'][0:90])
    return vectorizer_claim, vectorizer_body

def fc(vectorizer_claim, vectorizer_body, data):
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50), random_state=1)
    
    clf.fit(train_data, data['label'][i:i+1000])
    X_test = vectorize.transform(data['X'][15000:15555])
    y_test = data['label'][15000:15555]
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))


if __name__ == '__main__':
    vectorizer_claim, vectorizer_body = to_tdidf()
    
    # data = pd.read_pickle(os.path.join(MY_HOME, "preprocessed_claim_body.pkl"))
    # body = vectorizer_claim.transform(data['body'])
    # claim = vectorizer_claim.transform(data['claim'])
    # prepare_batch(vectorizer_claim, vectorizer_body, data)


    # print(data.head())


==========> Data Preprocessing
  label  ...                                               body
0     0  ...   on thousand  nin hundr and eightyfour georg o...
1     2  ...   republ who crit parkland stud drop out of mai...
2     1  ...   first person to complet nas s passport progra...
3     2  ...   how dang be measl tweet twentyseven march two...
4     2  ...    remark on counterter at stanford univers slo...

[5 rows x 3 columns]


In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(1001, 100)
        self.fc2 = nn.Linear(100, 3)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(x)
        return x
mlp = Net()


In [0]:
from torch.utils.data import Dataset, DataLoader
import torch

class ClaimBodyDataset(Dataset):
    """claim body dataset."""
    # TODO: check using loading the dataframe directly
    def __init__(self, vectorizer_claim, vectorizer_body, df="drive/My Drive/nlp/preprocessed_claim_body.pkl"):
        """
        Args:
            claim: 5000 by 1
            body: 5000 by 1
            cosine_similarity: 1 by 1
        """
        self.df = pd.read_pickle(df)
        self.vectorizer_claim = vectorizer_claim.transform(self.df['claim'])
        self.vectorizer_body = vectorizer_body.transform(self.df['body'])
        self.cosine_similarity = cosine_similarity

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        claim_tfidf = self.vectorizer_claim[i]
        body_tfidf = self.vectorizer_body[i]
        cos_tfidf = cosine_similarity(body_tfidf, claim_tfidf)
        feat_vec = np.squeeze(np.c_[claim_tfidf.toarray().reshape(1, -1), body_tfidf.toarray().reshape(1, -1), cos_tfidf])
        label = self.df['label'].iloc[idx]

        feat_vec = torch.from_numpy(feat_vec)
        label = torch.tensor([label], dtype=torch.long)

            
        return feat_vec, label



In [145]:
import torchvision.transforms as transforms

claim_body_dataset = ClaimBodyDataset(vectorizer_claim, vectorizer_body)


for i in range(4):
    sample = claim_body_dataset[i]
    print(i, sample)


0 (tensor([0.0000, 0.0000, 0.0000,  ..., 0.0142, 0.0360, 0.0112],
       dtype=torch.float64), tensor([0]))
1 (tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0080, 0.0361],
       dtype=torch.float64), tensor([2]))
2 (tensor([0.0000, 0.0000, 0.0000,  ..., 0.0067, 0.1403, 0.0303],
       dtype=torch.float64), tensor([1]))
3 (tensor([0.0000, 0.0000, 0.0000,  ..., 0.0258, 0.0541, 0.0226],
       dtype=torch.float64), tensor([2]))


In [31]:
!pwd

/content


In [0]:
import torch.optim as optim
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.001, momentum=0.9)
dataset_loader = torch.utils.data.DataLoader(claim_body_dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=4)

In [147]:


def train_mlp(trainloader, criterion, optimizer):

    for epoch in range(2):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = mlp(inputs.float())
            outputs = outputs.reshape(1, 3)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')

train_mlp(claim_body_dataset, criterion, optimizer)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


Finished Training


In [173]:
def accuracy(model, data_x, data_y):
    model = model.eval() 

    X = data_x
    Y = torch.LongTensor(data_y)
    Y = Y.T
    output = model(X.float())
    _, predicted = torch.max(output.data, 1)
    
    # arg_maxs is tensor of indices [0, 1, 0, 2, 1, 1 . . ]
    num_correct = torch.sum(Y==predicted)
    acc = (num_correct * 100.0 / len(data_y))
    print(Y)

    return acc.item()  # percentage format


tensor([[1, 0, 0, 1]])
50.0
tensor([[1, 0, 1, 0]])
50.0
tensor([[1, 1, 0, 1]])
75.0
tensor([[0, 0, 0, 1]])
25.0
tensor([[0, 2, 0, 0]])
0.0
tensor([[1, 0, 0, 1]])
50.0
tensor([[0, 0, 2, 1]])
25.0
tensor([[0, 1, 0, 0]])
25.0
tensor([[0, 2, 0, 0]])
0.0
tensor([[1, 1, 1, 0]])
75.0
tensor([[0, 2, 2, 1]])
25.0
tensor([[1, 1, 1, 0]])
75.0
tensor([[1, 1, 2, 0]])
50.0
tensor([[2, 1, 1, 0]])
50.0
tensor([[1, 1, 0, 1]])
75.0
tensor([[2, 1, 0, 0]])
25.0
tensor([[2, 0, 0, 1]])
25.0
tensor([[1, 0, 1, 0]])
50.0
tensor([[0, 2, 0, 1]])
25.0
tensor([[0, 1, 1, 2]])
50.0
tensor([[2, 1, 1, 1]])
75.0
tensor([[1, 2, 0, 2]])
25.0
tensor([[1, 0, 1]])
66.66666412353516


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [0]:
# NOTE:this only tests on the original training set
# need to figure out this train vs test vectorizer fit issue
# see https://github.com/uclnlp/fakenewschallenge/blob/badc65fb8d06b840ab25093ea5fdd4bc3a365c30/pred.py#L53
for i, data in enumerate(dataset_loader, 0):
    inputs, labels = data
    print(accuracy(mlp, inputs, labels))

In [19]:
#!ls drive/My\ Drive
MY_HOME = 'drive/My Drive/' #metadata_articles_dataframe.pkl
!ls drive

'My Drive'


In [0]:
import pandas as pd
import os
import string
data = pd.read_pickle(os.path.join(MY_HOME, "nlp/preprocessed_training_dataframe.pkl"))
# define documents
docs = data['X'][0:12000]



In [0]:
# TODO: find a claim 
# cut off at the first period
# use cosine similarity TFIDF

 russ build a monu to mem the genocid of indig peopl on the be contin outsid the unit stat embassy in russ . ind kil in be genocid mem in moscow ind on of the suggest im for a propos be ind mem in front of the be embassy in moscow the civ chamb of the russ feed rec annount a propos to build a mem commem the genocid of be ind in front of the be embassy in moscow accord to the the civ chamb of the russ feed rec annount a propos to build a mem commem the genocid of be ind in front of the be embassy in moscow accord to the observ the civ chamb of russ feed be comp to yo hous commit on oversight in the yo civ chamb memb valery korovin tell the observ thi monu must becom the sil reproach of the modern be elit which have sign devy from the id principl that wer lay into the find of the be stat although the feed be plan to forward the propos to presid vladimir putin s admin the form of and fund for the mem ar vagu vladimir larin a video blog in moscow and a hand of friend get the ide for the me

In [22]:
import string
str = docs[349]
print(docs[349])

def get_claim_and_body(str):
  """
  separates the body and the claim in a single entry
  """
  summary = []
  for i, char in enumerate(str):
    if char != '.':
      summary.append(char)
      
    elif i > 0.2 * len(str):
      break
    elif char == '.':
      break
  summary = convert(summary)
  summary = summary.translate(str.maketrans('', '', string.punctuation))

  body = str[i:len(str)]
  body = body.translate(str.maketrans('', '', string.punctuation))

  return summary, body

def convert(s): 
  
  # initialization of string to "" 
  str1 = "" 
  
  # using join function join the list s by  
  # separating words by str1 
  return(str1.join(s)) 


def df_to_body_and_claim(df):
  

wayn lapier say an obama-er rul deny gun right to the eld who `` seek help to do their tax . '' bid rev nra hist on background check vic presid joe bid exag when he wax nostalg about the good old day a tim when everybody includ the nra think background check mad sens bid s off say he be refer to the nra s support for background check in the ear 1990s and it stat support for expand background check to includ gun show in on thousand , nin hundr and ninety-nine but vic presid s senty journey rid roughshod ov the fact in on thousand , nin hundr and ninety-one the nra endors legisl cre a nat system of inst background check as an altern to a sevenday wait period contain in the propos brady handgun viol prev act but the technolog do not ex yet a guncontrol advoc say the inst check bil would hav complet gut the brady bil by elimin the wait period in on thousand , nin hundr and ninety-three the brady bil becam law with comprom langu on background check the law requir a fiveday wait period until

# KERAS to try the word embedding method
- [ ] separate them into proper functions
- [ ] try to use the FC and fit_transform of SKLEARN TFIDF

In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
import pandas as pd
import os

data = pd.read_pickle(os.path.join(MY_HOME, "nlp/preprocessed_training_dataframe.pkl"))
# define documents
docs = data['X'][0:12000]
# define class labels
labels = data['label'][0:12000]
print(labels)
# convert labels to onehot
labels = one_hot(labels, 3)
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
# pad documents to a max length of 300 words
max_length = 20
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post', truncating='post')
# load the whole embedding into memory
embeddings_index = dict()
f = open(os.path.join(MY_HOME, 'nlp/glove.6B.50d.txt'))
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 50))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=max_length, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(32))
model.add(Dense(3, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m,precision_m, recall_m])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=100, verbose=0)


id
0        0
1        2
4        1
5        2
6        2
        ..
13211    2
13212    0
13213    1
13214    1
13215    0
Name: label, Length: 12000, dtype: int64
Loaded 400000 word vectors.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 50)            16058550  
_________________________________________________________________
flatten_2 (Flatten)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                32032     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 99        
Total params: 16,090,681
Trainable params: 32,131
Non-trainable params: 16,058,550
_________________________________________________________________
None


In [0]:
# evaluate the model
# define documents
docs = data['X'][12000:15555]
# define class labels
labels = data['label'][12000:15555]
print(labels)
# convert labels to onehot
labels = one_hot(labels, 3)
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
# pad documents to a max length of 4 words
max_length = 20
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

loss, accuracy, f1_score, precision, recall = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))
print("f1_score, precision, recall", f1_score, precision, recall)

id
13216    1
13217    0
13218    1
13219    2
13220    0
        ..
17137    2
17138    0
17139    0
17140    2
17141    1
Name: label, Length: 3555, dtype: int64
Accuracy: 63.769340
f1_score, precision, recall 0.36785523177199225 0.43943006579886007 0.3181434599156118


In [0]:
from sklearn.metrics import classification_report
y_pred = model.predict_classes(padded_docs, batch_size=64, verbose=1)
# y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(np.argmax(labels, axis=1), y_pred))

3555/3555 [==============================] - 0s 18us/step
              precision    recall  f1-score   support

           0       0.46      0.53      0.49      1638
           1       0.43      0.45      0.44      1530
           2       0.10      0.02      0.03       387

    accuracy                           0.44      3555
   macro avg       0.33      0.33      0.32      3555
weighted avg       0.41      0.44      0.42      3555

